In [3]:
import pandas as pd

In [5]:
test_path = "/home/monlamai/Documents/GitHub/combining-dataset/550-test.tsv"
df = pd.read_csv(test_path, sep='\t')

In [6]:
df.head()

,file_name,uni,wylie,url,dept
0,STT_TT00009_02069.100-02070.650,བྱམས་པ་དང་བརྩེ་བ་དཔེ་ཚད་,byams pa dang brtse ba dpe tshad _,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT
1,STT_TT00010_01281.050-01282.650,སངས་རྒྱས་ཆོས་དང་དགེ་འདུན་ནི།།ཐར་པ་འདོད་པ་རྣམས་...,sangs rgyas chos dang dge 'dun ni//thar pa 'do...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT
2,STT_NS0018_0242_1748899_to_1754434,དེ་དུས་པུ་གུ་ར་དེ་དུས་འུ་འུ་པུ་གུ་ཡིན་དུས་སྐབས...,de dus pu gu ra de dus 'u 'u pu gu yin dus ska...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS
3,STT_TT00377_601.800-603.750,ཅིག་མི་རྫོགས་རྒྱུ་མེད་པ་ཞིག་བཤད་རྩ་རུ་མ་བཤད་པར...,cig mi rdzogs rgyu med pa zhig bshad rtsa ru m...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT
4,STT_TT00380_547.200-548.900,ད་ཁྱོན་ནས་གསོད་དགོས་མིན་འགྲོ་བསམ་བསམ་དེ་འདྲའི་གི,da khyon nas gsod dgos min 'gro bsam bsam de '...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT


In [14]:
from transformers import pipeline
import torch

if torch.cuda.is_available():
    device = "cuda:0"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

generator = pipeline(
    "automatic-speech-recognition",
    model="openpecha/wav2vec2_run8",
)

In [15]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset

dataset = Dataset.from_pandas(df)
all_predictions = []

# run streamed inference
for prediction in tqdm(
    generator(
        KeyDataset(dataset, "url"),
        max_new_tokens=128,
        generate_kwargs={"task": "transcribe"},
        batch_size=32,
    ),
    total=len(dataset),
):
    all_predictions.append(prediction["text"])

  0%|          | 64/27009 [01:21<9:31:11,  1.27s/it] 


KeyboardInterrupt: 

In [ ]:
from evaluate import load

cer_metric = load("cer")

cer_ortho = 100 * cer_metric.compute(
    references=dataset["uni"], predictions=all_predictions
)
cer_ortho

In [6]:
# import os
from transformers import pipeline
from datasets import Dataset

generator = pipeline(model="openpecha/wav2vec2_run8", device="cuda:0")


# opt = generator(df.loc[0, 'url'])
# inf = opt['text']
# print(inf)

dataset = Dataset.from_pandas(df)

def generate(batch):
    
    urls = batch["url"]
    generated_texts = generator(urls)  # This should process all URLs in the batch at once
    # Extracting the 'text' field from each generated output and assigning it back to the batch
    batch['inf'] = [item['text'] for item in generated_texts]
    return batch

# df['inf']  = df.apply(lambda row: generator(row['url'])['text'], axis=1)
updated_dataset = dataset.map(generate, batched=True, batch_size=8)

Map:   0%|          | 0/27010 [00:00<?, ? examples/s]

/home/monlamai/Documents/GitHub/stt-whisper/.env/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/monlamai/Documents/GitHub/stt-whisper/.env/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/monlamai/Documents/GitHub/stt-whisper/.env/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/monlamai/Documents/GitHub/stt-whisper/.env/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
 

In [7]:
df_out = updated_dataset.to_pandas()

In [46]:
# import pyewts

# converter = pyewts.pyewts()

# df_out['inf_uni'] = df_out['inf'].map(lambda x : converter.toUnicode(x))

བ་བྦ་a་བ་


In [8]:
from evaluate import load

cer_metric = load("cer")

In [9]:
df_out

cer = cer_metric.compute(references=[df_out.loc[0, 'uni']], predictions=[df_out.loc[0, 'inf']])

print(cer)

0.5192307692307693


In [10]:
df_out['cer'] = df_out.apply(lambda row: min(cer_metric.compute(references=[df_out.loc[0, 'uni']], predictions=[df_out.loc[0, 'inf']]), 1), axis=1)

In [11]:
mean_cer = df_out['cer'].mean()

In [12]:
mean_cer

0.5192307692307693

In [13]:
mean_cer * 100

51.92307692307693

In [52]:
df_out.head()

,file_name,uni,wylie,url,dept,inf,wer,cer,inf_uni
0,STT_TT00334_0137_691554_to_694146,ད་བསམ་བློའི་ནང་མ་དྲན་དགུ་དྲན་དྲན། བ་ལོང་དགུ་ལོ...,da bsam blo'i nang ma dran dgu dran dran/_ba l...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,da bsam blo'i nang la ma dran dgu dran dran sb...,0.500000,0.25,ད་བསམ་བློའི་ནང་ལ་མ་དྲན་དགུ་དྲན་དྲན་སྦ་ལམ་སྔོན་...
1,STT_TT00103_10786.950-10789.300,དེ་དང་ལྔ། ནོར་བུ་རིན་ཆེན་དེ་དང་དྲུག,de dang lnga/_nor bu rin chen de dang drug_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,de dang lnga//nor bu rin chen de dang drug//,0.222222,0.25,དེ་དང་ལྔ༎ནོར་བུ་རིན་ཆེན་དེ་དང་དྲུག༎
2,STT_NS0024_0201_1564675_to_1574645,སྔོན་མ་ང་ཚོ་དཔའ་དཔལ་རྡོར་གྲོངས་ངེས་ཡོད་པ་དཔའ་ད...,sngon ma nga tsho dpa' dpal rdor grongs nges y...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_NS,sngon ma nga tsho dpa' dpal rdor grong nges yo...,0.578125,0.25,སྔོན་མ་ང་ཚོ་དཔའ་དཔལ་རྡོར་གྲོང་ངེས་ཡོད་བ་དཔག་པའ...
3,STT_TT00213_00473.000-00476.500,བསམ་བློའི་ཆུ་ཚོད་ཟེར་ནའང་མི་འདུག ནམ་རྒྱུན་སོ་ས...,bsam blo'i chu tshod zer na'ang mi 'dug_nam rg...,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,bsam blo'i chu tshad zer myi me tog nam rgyun ...,0.521739,0.25,བསམ་བློའི་ཆུ་ཚད་ཟེར་མྱི་མེ་ཏོག་ནམ་རྒྱུན་སོ་སོས...
4,STT_TT00086_2971.050-2972.250,སྤྱི་བའི་དོན་དག་ལ་ཡ།,spyi ba'i don dag la ya/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,spyi ba'i don dag la ya,0.166667,0.25,སྤྱི་བའི་དོན་དག་ལ་ཡ་


In [56]:

df_out.to_csv('test_wav2vec2.tsv', sep='\t', index=False)